# TP4 Supervised learning: Regression problem (Correction)

In [ ]:
pip install scikit-optimize

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Part 1: Descriptive statistics and preprocessing

1) Load _*train.csv*_ and _*test.csv*_ datasets, print their shapes, display the first few rows, and provide a summary with [pandas.DataFrame.describe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html) for each dataset.

In [ ]:
# Answer
import pandas as pd
train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")

print("train shape: ", train_data.shape)
print("test shape: ", test_data.shape)

In [ ]:
print("Train head:")
display(train_data.head(3))
print("Test head:")
test_data.head(3)

In [ ]:
print("train dataset description:")
display(train_data.describe())

In [ ]:
print("test dataset description:")
display(test_data.describe())

2) Extract ```SalePrice``` as the target variable from the ```train``` and ```test``` datasets, storing them as ```train_target``` and ```test_target``` respectively. Remove unnecessary variables from the same datasets.

In [ ]:
# Answer
train_target = train_data["SalePrice"]
test_target = test_data["SalePrice"]
train_data.drop(["Id", "SalePrice"], axis = 1, inplace=True)
test_data.drop(["Id", "SalePrice"], axis = 1, inplace=True)

# Verify that the ID and SalePrice variable are correctly deleted
print("Train head:")
display(train_data.head(3))
print("Test head:")
display(test_data.head(3))

3) Define a function that identifies variables with missing values, and returns each variable's name, the number of missing values, and the percentage of missing values.

In [ ]:
# Answer
def missing_values(data):
    missing_values = data.isna().sum().sort_values(ascending=False)
    n_missing_values = missing_values[missing_values>0]
    p_missing_values = (n_missing_values/data.shape[0])*100
    missing_data = pd.concat([n_missing_values,p_missing_values], axis=1, keys=['Count', 'Percentage'])
    return missing_data

print("Missing values of train data:")
display(missing_values(train_data))
print(" ")
print("Missing values of test data:")
display(missing_values(test_data))

4) For simplicity, fill the missing values with 0 using [pandas.DataFrame.fillna](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html) (modify the same dataset).

In [ ]:
# Answer
train_data.fillna(0.0, inplace=True)
test_data.fillna(0.0, inplace=True)

# Verify that the missing values are well filled (i.e., there's no more missing values)
print("Missing values of train data:")
print(missing_values(train_data))
print(" ")
print("Missing values of test data:")
print(missing_values(test_data))

5) Describe the target variable ```train_target```

In [ ]:
# Answer
print(train_target.describe())
print("Skewness: ", train_target.skew())
print("Kurtosis: ", train_target.kurtosis())

6) Plot the histogram and density of ```train_target``` (you can use [seaborn.displot](https://seaborn.pydata.org/generated/seaborn.displot.html) module)

In [ ]:
# Answer
import matplotlib.pyplot as plt
import seaborn as sns
sns.displot(train_target,bins=30, kde = True, stat="density", color = 'darkblue')
plt.title('Histogram of SalePrice')
plt.show()

7) Plot histograms of all other variables using [pandas.DataFrame.hist](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.hist.html)

In [ ]:
# Answer
train_data.hist(figsize=(20, 20), bins=30, layout=(7, 6));

8) Compute and plot the correlation matrix between the variables using [seaborn.heatmap](https://seaborn.pydata.org/generated/seaborn.heatmap.html). Comment the results.

In [ ]:
# Answer
correlation_matrix = pd.concat([train_data,train_target], axis=1).corr()
plt.figure(figsize = (10,8))
sns.heatmap(correlation_matrix);

9) Visualize correlations between ```SalePrice``` and other variables using a [seaborn.barplot](https://seaborn.pydata.org/generated/seaborn.barplot.html).

In [ ]:
# Answer
sale_price_correlation = correlation_matrix.loc['SalePrice', :]
sale_price_correlation = sale_price_correlation.sort_values(ascending=False)
fig = plt.figure(figsize=(12, 8))
sns.barplot(x=sale_price_correlation, y=sale_price_correlation.index, orient='h', palette='flare')
plt.xlabel('Correlation')
plt.title('Correlation of SalePrice with Other Features')
plt.show()
fig.autofmt_xdate()

10) Visualize the scatter plot of the ```SalePrice``` variable as a function of the ```GrLivArea```. Comment.

In [ ]:
# Answer
plt.figure(figsize = (12,8))
sns.scatterplot(data = pd.concat([train_data, train_target], axis=1), x = "GrLivArea", y = "SalePrice");

11) Visualize the boxplot of the ```SalePrice``` variable as a function of the ```OverallQual``` using [seaborn.boxplot](https://seaborn.pydata.org/generated/seaborn.boxplot.html). Interpret the boxplot

In [ ]:
# Answer
plt.figure(figsize = (12,8))
sns.boxplot(data=pd.concat([train_data, train_target], axis=1), x="OverallQual", y = "SalePrice");

12) Visualize the empirical distributions of the train and test dataset (for some variables). Comment the results.

In [ ]:
# Answer
n_cols = 3
fig = plt.figure(figsize=(3*n_cols,6))
for j, i in enumerate(train_data.columns[:n_cols]):
    plt.subplot(310+j+1)
    sns.distplot(train_data[i], label = 'train')
    sns.distplot(test_data[i], label = 'test')
    plt.legend()
fig.tight_layout()

### Part 2: Train and evaluate models

13) Split the data into training and validation data using [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [ ]:
# Answer
from sklearn.model_selection import train_test_split
tr_x, val_x, tr_y, val_y  = train_test_split(train_data, train_target, shuffle=True, random_state=42, test_size=0.20)

14) Fit a [linear regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) (we choose the linear regression to learn to predict the target variable) and measure its performance using the [RMSE](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) (RMSE = Root Mean Square Error) and [MAPE](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_percentage_error.html) (MAPE = Mean Absolute Percentage Error) as metric. Measure its performance on the test data. Comment the results.

**Recall:**

$$RMSE(Y,\hat{Y}) = \sqrt{\frac{1}{n} \sum_{i=1}^n (Y_i - \hat{Y_i})^2}\,\,\,\,\,\,\,\,MAPE(Y,\hat{Y}) = \frac{1}{n} \sum_{i=1}^n \frac{|Y_i - \hat{Y_i}|}{Y_i}$$

where $Y$ is the true target and  $\hat{Y}$ is the predicted target  

In [ ]:
# Answer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

model = LinearRegression()
model.fit(tr_x, tr_y)
val_pred_y = model.predict(val_x)

rmse = mean_squared_error(val_y,val_pred_y, squared=False)
print("RMSE on validation set: ", rmse)
test_pred_y = model.predict(test_data)
rmse = mean_squared_error(test_target,test_pred_y, squared=False)
print("RMSE on test set: ", rmse)

mape = mean_absolute_percentage_error(val_y,val_pred_y)
print("MAPE on validation set: ", mape)
mape = mean_absolute_percentage_error(test_target,test_pred_y)
print("MAPE on test set: ", mape)

**Objective: Improve the predictions!**

15) Train the following models and evaluate their performance on validation and test set.
* [K-Nearest Neighbors Regressor (KNeighborsRegressor)](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)
* [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)
* [Random Forest Regressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)

In [ ]:
# Answer
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import  Ridge
from sklearn.ensemble import RandomForestRegressor

# Complete your answer here

# Answer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

model = RandomForestRegressor()
model.fit(tr_x, tr_y)
val_pred_y = model.predict(val_x)

rmse = mean_squared_error(val_y,val_pred_y, squared=False)
print("RMSE on validation set: ", rmse)
test_pred_y = model.predict(test_data)
rmse = mean_squared_error(test_target,test_pred_y, squared=False)
print("RMSE on test set: ", rmse)

mape = mean_absolute_percentage_error(val_y,val_pred_y)
print("MAPE on validation set: ", mape)
mape = mean_absolute_percentage_error(test_target,test_pred_y)
print("MAPE on test set: ", mape)

16) Define a function that takes in parameter a dictionnary of models and returns the mean and standard deviation of the MAPE on [cross validation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) with three sets. You can consider the following models:

* [Support Vector Regressor (SVR)](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html)
* [Extreme Gradient Boosting Regressor (XGBRegressor)](https://xgboost.readthedocs.io/en/stable/parameter.html)
* [Decision tree regressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)
* [K-Nearest Neighbors Regressor (KNeighborsRegressor)](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)
* [Linear Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
* [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)
* [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)
* [Gradient Boosting Regressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html)
* [Random Forest Regressor](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)

In [ ]:
# Answer
import numpy as np
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer

def evaluate(models, X_train, y_train, metric):
    for model in models:
        models[model]['score'] = cross_val_score(models[model]['model'], X_train, y_train, cv=3,scoring=metric)
        print("{} : {} (+/- {})".format(models[model]['name'], models[model]['score'].mean(), models[model]['score'].std()**2))

MAPE = make_scorer(mean_absolute_percentage_error)

In [ ]:
models = {'gbc': {'model': GradientBoostingRegressor(), 'name': 'GradientBoostingRegressor'},
          'xgb': {'model': XGBRegressor(), 'name': 'XGBRegressor'},
          'rf': {'model': RandomForestRegressor( n_jobs=-1), 'name':'RandomForestRegressor'},
          'tree': {'model': DecisionTreeRegressor(), 'name':'DecisionTreeRegressor'},
          'svr': {'model': SVR(), 'name': 'SVR'},
          'knn': {'model': KNeighborsRegressor(), 'name': 'KNeighborsRegressor'},
          'lr': {'model': LinearRegression(), 'name': 'LinearRegression'},
          'ridge': {'model': Ridge(), 'name': 'Ridge'},
          'lasso': {'model': Lasso(), 'name': 'Lasso'}
         }
evaluate(models,  train_data, train_target, metric = MAPE)

### Part 4: Fine tunning

17) Choose three best methods and evaluate their performance on the test data by varying some of their parameters.

In [ ]:
# Answer
model = GradientBoostingRegressor(n_estimators=200,learning_rate = 0.1, max_depth=3).fit(train_data,train_target)
pred_target = model.predict(test_data)
print ('GradientBoostingRegressor: {}'.format(mean_absolute_percentage_error(test_target,pred_target)))

In [ ]:
# Answer
model = GradientBoostingRegressor(n_estimators=150, max_depth=5).fit(train_data,train_target)
pred_target = model.predict(test_data)
print ('GradientBoostingRegressor: {}'.format(mean_absolute_percentage_error(test_target,pred_target)))

model = RandomForestRegressor(n_estimators=200, max_depth=8).fit(train_data,train_target)
pred_target = model.predict(test_data)
print ('RandomForestRegressor: {}'.format(mean_absolute_percentage_error(test_target,pred_target)))

model = XGBRegressor(n_estimators=300 , max_depth =4 ).fit(train_data,train_target)
pred_target = model.predict(test_data)
print ('XGBRegressor: {}'.format(mean_absolute_percentage_error(test_target,pred_target)))

18) Perform an automated parameter search using [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) on a model of your choice and assess the performance of the best-tuned model on the test dataset.

In [ ]:
# Answer
from sklearn.model_selection import GridSearchCV

model = GradientBoostingRegressor()

params = {
    'n_estimators' : [100, 300, 400],
    'max_depth' : [2, 4, 8],
    'learning_rate':[0.1, 0.05, 0.01]}

grid = GridSearchCV(model,param_grid=params,cv=3, scoring=MAPE, n_jobs=-1, verbose = 1 )
grid.fit(train_data, train_target)

In [ ]:
print( 'Grid search results: {}'.format(grid.best_params_))

# Return the best model:
best = grid.best_estimator_

print ('MAPE of the best model on test data:', mean_absolute_percentage_error(test_target,best.predict(test_data)))
print ('RMSE of the best model on test data:', mean_squared_error(test_target,best.predict(test_data), squared=False))

19) Perform an automated parameter search using Bayesian optimization with [gp_minimize](https://scikit-optimize.github.io/stable/modules/generated/skopt.gp_minimize.html) and assess the performance of the best-tuned model on the test dataset.

In [ ]:
# Answer
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args

space  = [Integer(1, 5, name='max_depth'),
          Real(10**-5, 10**0, "log-uniform", name='eta'),
          Real(10**-5, 10**0, "log-uniform", name='subsample')
          ]

model = XGBRegressor(n_estimators = 300)

@use_named_args(space)
def objective(**params):
    model.set_params(**params)

    return np.mean(cross_val_score(model, train_data, train_target, cv=4, n_jobs=-1,
                                    scoring=MAPE))
res_gp = gp_minimize(objective, space, n_calls=50, random_state=42, verbose=1)

In [ ]:
print("Best score = {}".format(res_gp.fun))
print("""Best params:
- max_depth = {}
- eta = {}
- subsample = {}
""".format(res_gp.x[0],res_gp.x[1],res_gp.x[2]))

In [ ]:
best_model = XGBRegressor(n_estimators=300, max_depth = 4, eta=0.025603767171805257, subsample=0.14637689184566424)
best_model.fit(train_data, train_target)
pred_target = best_model.predict(test_data)
print("MAPE on the test dataset: ", mean_absolute_percentage_error(test_target, pred_target))
print("RMSE on the test dataset: ", mean_squared_error(test_target, pred_target, squared=False))

### Part 5: Ensemble modeling

19) **Aggregation:** Fit and predict the target using 4 best models. Then, aggregate the results using the mean and median. Evaluate the performances.

In [ ]:
# Answer

models = {'GradientBoosting': {'model': GradientBoostingRegressor(n_estimators= 300), 'name': 'GradientBoostingRegressor'},
          'XGBoost': {'model': XGBRegressor(n_estimators=300), 'name': 'XGBRegressor'},
          'RandomForest': {'model': RandomForestRegressor(n_estimators=300, n_jobs=-1), 'name':'RandomForestRegressor'},
          'Ridge': {'model': Ridge(), 'name': 'Ridge'}
         }
predictions = {}
for model_name in models:
    model = models[model_name]["model"].fit(train_data, train_target)
    predictions[model_name] = model.predict(test_data)

In [ ]:
print ('''Aggregation by mean
RMSE = {}
MAPE = {}
'''.format(mean_squared_error(test_target,np.mean(list(predictions.values()),axis=0),squared=False),mean_absolute_percentage_error(test_target,np.mean(list(predictions.values()),axis=0))))

print ('''Aggregation by median
RMSE = {}
MAPE = {}
'''.format(mean_squared_error(test_target,np.median(list(predictions.values()),axis=0),squared=False),mean_absolute_percentage_error(test_target,np.median(list(predictions.values()),axis=0))))


20) **Stacking:** Perform the following steps:<br>

   1. Fit the 4 best models on the ```tr_x``` and save the predictions on ```val_x``` on a new dataframe named ```design_layer1``` and the predictions on ```data_test``` on ```test_layer1```. <br>
   2. Fit a new model on the ```design_layer1```. <br>
   3. Predict the target using the new model.  

In [ ]:
# Answer
models = {'GradientBoosting': {'model': GradientBoostingRegressor(n_estimators= 300), 'name': 'GradientBoostingRegressor'},
          'XGBoost': {'model': XGBRegressor(n_estimators=300), 'name': 'XGBRegressor'},
          'RandomForest': {'model': RandomForestRegressor(n_estimators=300), 'name':'RandomForestRegressor'},
          'Ridge': {'model': Ridge(), 'name': 'Ridge'}
         }
test_layer1 = pd.DataFrame()
design_layer1 = pd.DataFrame()
for model_name in models:
    model_layer1 = models[model_name]["model"].fit(tr_x, tr_y)
    design_layer1[model_name] = model_layer1.predict(val_x)
    test_layer1[model_name] = model_layer1.predict(test_data)
display(design_layer1.head())
display(test_layer1.head())

In [ ]:
modele_layer2 = LinearRegression()
modele_layer2.fit(design_layer1, val_y)
pred_target = modele_layer2.predict(test_layer1)

In [ ]:
print('RMSE = ', mean_squared_error(test_target,pred_target, squared=False))
print('MAPE = ', mean_absolute_percentage_error(test_target,pred_target))